<a href="https://colab.research.google.com/github/mohameddosama22/my-first-repo/blob/main/Internship%20Saiket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display


candidates = glob.glob("*.csv") + glob.glob("data/*.csv") + glob.glob("../*.csv")
telco_files = [p for p in candidates if 'telco' in os.path.basename(p).lower() or 'Telco' in os.path.basename(p)]
if len(telco_files) == 0:


    filepath = "Telco_Customer_Churn_Dataset .csv"
else:
    filepath = telco_files[0]

print("Using file:", filepath)
df = pd.read_csv(filepath)
print("Shape:", df.shape)
display(df.head())




Using file: Telco_Customer_Churn_Dataset .csv
Shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [14]:
print("Info:")
display(df.info())

print("\nMissing values per column:")
display(df.isna().sum())



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   ob

None


Missing values per column:


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [15]:
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].astype(str).str.strip(), errors='coerce')

    mask = df['TotalCharges'].isna()
    print("NaN in TotalCharges before imputation:", mask.sum())
    if mask.sum() > 0:
        df.loc[mask, 'TotalCharges'] = df.loc[mask, 'MonthlyCharges'] * df.loc[mask, 'tenure']


    df['TotalCharges'] = df['TotalCharges'].fillna(0)



if 'Churn' in df.columns:
    df['Churn_num'] = df['Churn'].map({'Yes':1, 'No':0})
    print("Churn counts:")
    display(df['Churn'].value_counts(dropna=False))
else:
    print("No 'Churn' column found.")


if 'SeniorCitizen' in df.columns:
    df['SeniorCitizen'] = pd.to_numeric(df['SeniorCitizen'], errors='coerce').fillna(0).astype(int)



binary_map = {'Yes':1, 'No':0, 'No internet service':0, 'No phone service':0}
candidate_cols = []
for c in df.columns:
    if df[c].dtype == object:
        vals = set([str(x).strip() for x in df[c].dropna().unique()])


        if vals <= set(['Yes','No','No internet service','No phone service']):
            candidate_cols.append(c)
print("Binary-like columns detected:", candidate_cols)
for c in candidate_cols:
    df[c] = df[c].replace(binary_map)


NaN in TotalCharges before imputation: 11
Churn counts:


,count
Churn,
No,5174
Yes,1869


Binary-like columns detected: ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']


/tmp/ipython-input-2508041571.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace(binary_map)


In [16]:
if 'customerID' in df.columns:
    df.drop(columns=['customerID'], inplace=True)


obj_cols = df.select_dtypes(include='object').columns.tolist()
print("Remaining object columns to encode:", obj_cols)


df_encoded = pd.get_dummies(df, columns=obj_cols, drop_first=True)


cleaned_path = "Telco_cleaned.csv"
df_encoded.to_csv(cleaned_path, index=False)
print("Saved cleaned file to:", cleaned_path)
print("Cleaned shape:", df_encoded.shape)
display(df_encoded.head())


Remaining object columns to encode: ['gender', 'InternetService', 'Contract', 'PaymentMethod']
Saved cleaned file to: Telco_cleaned.csv
Cleaned shape: (7043, 25)


,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,Churn,Churn_num,gender_Male,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,0,0,1,0,0,...,0,0,False,False,False,False,False,False,True,False
1,0,0,0,34,1,0,1,0,1,0,...,0,0,True,False,False,True,False,False,False,True
2,0,0,0,2,1,0,1,1,0,0,...,1,1,True,False,False,False,False,False,False,True
3,0,0,0,45,0,0,1,0,1,1,...,0,0,True,False,False,True,False,False,False,False
4,0,0,0,2,1,0,0,0,0,0,...,1,1,False,True,False,False,False,False,True,False
